<a href="https://colab.research.google.com/github/oliviasteeed/ChefGPT/blob/main/trying_LLM_fine_tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# olivia attempt llm fine tuning

In [2]:
!pip3 install transformers datasets torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.9/30.9 MB 2.9 MB/s eta 0:00:00

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip3 install --upgrade pip


In [15]:
import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel, Trainer, TrainingArguments
from datasets import load_dataset, Dataset

In [16]:
# IMPORT DATA

# If your data is in a CSV file:
# from datasets import Dataset

# Replace this with the path to your dataset
data = Dataset.from_csv("/Users/oliviasteed/Desktop/full_dataset copy.csv")

In [17]:
# Assuming 'column_name1' and 'column_name2' are the columns you want to remove
data = data.remove_columns(['link', 'source', 'Unnamed: 0'])

In [ ]:
# SPLIT DATASET INTO TEST AND TRAIN

# Split dataset into training and validation sets (80% for training, 20% for validation)
train_test_split = data.train_test_split(test_size=0.2)

# Create training and validation datasets
train_dataset = train_test_split['train']
val_dataset = train_test_split['test']

In [18]:
data

Dataset({
    features: ['title', 'ingredients', 'directions', 'NER'],
    num_rows: 2231142
})

In [23]:
# PREPROCESS THE DATA

tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

# Add a padding token to the tokenizer
tokenizer.pad_token = tokenizer.eos_token

# Tokenize the entire dataset
def tokenize_function(examples):

  text = [f"{title} {ingredients} {directions}" for title, ingredients, directions in zip(examples["title"], examples["ingredients"], examples["directions"])]

    # Tokenize the concatenated text
  return tokenizer(text, padding="max_length", truncation=True)


    # return tokenizer(examples["title", "ingredients", "directions"], padding="256", truncation=True)

tokenized_data = data.map(tokenize_function, batched=True)

Map:   0%|          | 0/2231142 [00:00<?, ? examples/s]

In [24]:
# LOAD GPT MODEL

model = GPT2LMHeadModel.from_pretrained("gpt2")
model.resize_token_embeddings(len(tokenizer))  # Resize if adding custom tokens

Embedding(50257, 768)

In [26]:
!pip3 install transformers[torch]
!pip3 install accelerate -U

zsh:1: no matches found: transformers[torch]

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip3 install --upgrade pip


In [27]:
# SET UP TRAINING ARGUMENTS

training_args = TrainingArguments(
    output_dir="./gpt2_recipe_model",  # Directory to save the model
    overwrite_output_dir=True,
    num_train_epochs=5,  # Adjust the number of epochs
    per_device_train_batch_size=2,  # Adjust the batch size based on your GPU
    logging_dir="./logs",
    logging_steps=200,
    save_steps=500,
    save_total_limit=2,
    eval_steps=500,  # Evaluate on validation set every 500 steps
    evaluation_strategy="steps",  # Evaluate on validation set during training
)

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


ImportError: Using the `Trainer` with `PyTorch` requires `accelerate>=0.26.0`: Please run `pip install transformers[torch]` or `pip install 'accelerate>={ACCELERATE_MIN_VERSION}'`

In [ ]:
# TRAIN MODEL

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_data,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
)

trainer.train()

In [ ]:
# SAVE FINE TUNED MODEL

model.save_pretrained("./gpt2_recipe_model")
tokenizer.save_pretrained("./gpt2_recipe_model")

In [ ]:
# GENERATE RECIPES WITH MODEl

# Load the fine-tuned model
model = GPT2LMHeadModel.from_pretrained("./gpt2_recipe_model")
tokenizer = GPT2Tokenizer.from_pretrained("./gpt2_recipe_model")

# Function to generate a recipe based on ingredients
def generate_recipe(ingredients):
    prompt = f"Recipe: {ingredients}\nIngredients: {ingredients}\nInstructions:"
    inputs = tokenizer(prompt, return_tensors="pt")
    outputs = model.generate(inputs["input_ids"], max_length=200, num_return_sequences=1, no_repeat_ngram_size=2)
    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return generated_text

# Example usage:
ingredients = "spaghetti, eggs, pancetta, Parmesan, black pepper"
recipe = generate_recipe(ingredients)
print(recipe)